In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("proj2.ipynb")

# Project 2: Query Performance
## Due Date: Wednesday, October 9, 5:00 PM

## Assignment Details
In this project, we will explore how the database system optimizes query execution and how users can further tune the performance of their queries.

This project works with the [Lahman's Baseball Database](http://seanlahman.com/), an open source collection of baseball statistics from 1871 to 2020. It contains a variety of data, like batting statistics, team stats, managerial records, Hall of Fame records, and much more.

You may find this project's queries to be simpler than in Project 1. However, although the queries may not be as complex, we still expect you to spend ample time thinking through the effects of each of the methods, as reasoning about the tradeoff between different approaches is the goal of this assignment.

**Note:** If at any point during the project, the internal state of the database or its tables have been modified in an undesirable way (i.e. a modification not resulting from the instructions of a question), restart your kernel and clear output and simply re-run the notebook as normal. This will shutdown your current connection to the database, which will prevent the issue of multiple connections to the database at any given point, and when re-running the notebook you will create a fresh database based on the provided Postgres dump.

## Logistics & Scoring Breakdown

- Each coding question has **both public tests and hidden tests**. Roughly 50% of your coding grade will be made up of your score on the public tests released to you, while the remaining 50% will be made up of unreleased hidden tests.
- Public tests for multiple choice questions are for sanity check only (e.g. you are answering in the correct format). Partial credit will be awarded.
- Free-response questions will be manually graded. Please answer thoughtfully and concisely in complete sentences, drawing from knowledge in lectures and from your inspection of query plans.

This is an **individual project**. However, you’re welcome to collaborate with any other student in the class as long as it’s within the academic honesty guidelines.

**Please see this [spreadsheet](https://docs.google.com/spreadsheets/d/1UPcOGt4gQwHRm1i6pEDlpjNe-1iT-r_-t5IYRYW7YpQ/edit?usp=sharing) for the points breakdown.**

In [2]:
# Run this cell to set up imports
import numpy as np
import pandas as pd

## Getting Connected
Similar to Project 1, we will be using the `JupySQL` library to connect this notebook to a PostgreSQL database server on your JupyterHub account. Run the following cell to initiate the connection.

In [3]:
%reload_ext sql
%sql postgresql://jovyan@127.0.0.1:5432/postgres

In [4]:
# See full display
%config SqlMagic.displaylimit = 50

## Setting up the Database
The following cell will create the `baseball` database (if needed), unzip the Postgres dump of the Lahman's Baseball Database, populate the `baseball` database with the desired tables and data, and finally display all databases associated with the Postgres instance. After running the cell, you should see the `baseball` database in the generated list of databases outputted by `%sql \l`.

**Note:** If you run into the **role does not exist**/**database does not exist** error the first time you run this cell, feel free to ignore it. It does not affect data import.

In [5]:
!unzip -u data/baseball.zip -d data/

Archive:  data/baseball.zip


In [6]:
!psql postgresql://jovyan@127.0.0.1:5432/baseball -c 'SELECT pg_terminate_backend(pg_stat_activity.pid) FROM pg_stat_activity WHERE datname = current_database()  AND pid <> pg_backend_pid();'
!psql -h localhost -c 'DROP DATABASE IF EXISTS baseball'
!psql -h localhost -c 'CREATE DATABASE baseball'
!psql -h localhost -d baseball -f data/baseball.sql
!psql -h localhost -c 'SET max_parallel_workers_per_gather = 0;'

 pg_terminate_backend 
----------------------
(0 rows)

DROP DATABASE
CREATE DATABASE
SET
SET
SET
SET
SET
 set_config 
------------
 
(1 row)

SET
SET
SET
SET
SET
SET
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE
COPY 5219
COPY 104256
COPY 179
COPY 6236
COPY 425
COPY 6879
COPY 104324
COPY 13943
COPY 17350
COPY 138838
COPY 12028
COPY 31955
COPY 13110
COPY 4191
COPY 3040
COPY 3469
COPY 93
COPY 252
COPY 19370
COPY 45806
COPY 5445
COPY 26428
COPY 1207
COPY 325
COPY 2865
COPY 120
COPY 52
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALTER TABLE
ALT

In [7]:
%sql \l baseball

Running query in 'postgresql://jovyan@127.0.0.1:5432/postgres'

Name,Owner,Encoding,Collate,Ctype,Access privileges
baseball,jovyan,UTF8,en_US.utf8,en_US.utf8,None


Now, run the following cell to connect to the `baseball` database. There should be no errors after running the following cell.

In [8]:
%sql postgresql://jovyan@127.0.0.1:5432/baseball

Connecting and switching to connection postgresql://jovyan@127.0.0.1:5432/baseball

To ensure that the connection to the database has been established, let's try grabbing the first 5 rows from the `hall_of_fame` table.

In [9]:
%%sql
SELECT * FROM hall_of_fame LIMIT 5;

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

5 rows affected.

player_id,year_id,voted_by,ballots,needed,votes,inducted,category,needed_note
cobbty01,1936,BBWAA,226,170,222,Y,Player,None
ruthba01,1936,BBWAA,226,170,215,Y,Player,None
wagneho01,1936,BBWAA,226,170,215,Y,Player,None
mathech01,1936,BBWAA,226,170,205,Y,Player,None
johnswa01,1936,BBWAA,226,170,189,Y,Player,None


## Connect to the grader

Run the following cell for grading purposes.

In [10]:
# Just run the following cell, no further action is needed.
from data101_utils import GradingUtil
grading_util = GradingUtil("proj2")
grading_util.prepare_autograder()

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />


## Table Descriptions

In its entirety the [Lahman's Baseball Database](http://seanlahman.com/) contains 27 tables containing a variety of statistics for players, teams, games, schools, etc. For simplicity, this project will focus on a subset of the tables:

* `appearances`: details on the positions each player appeared at
* `batting`: batting statistics for each player
* `college_playing`: list of players and the colleges they attended
* `hall_of_fame`: Hall of Fame voting data
* `people`: player information (name, date of birth, and biographical info)
* `salaries`: player salary data
* `schools`: list of colleges that players attended

As a reminder from Project 1, the psql command `\d <table_name>` is helpful for identifying the schema of a table.

We **highly** encourage you open a Terminal in JupyterLab and connect to the database directly.

```sh
psql -h localhost -d baseball
```

...will drop you in a SQL interpreter. Remember you can use `\?` for help!


You may wish to review:
* The [Assignment Tips Guide](https://data101.org/fa24/assignment-tips/)
* The [Course Notes SQL Style Guide](https://data101.org/notes/appendix/sql-style.html)

For example the command below allows you to view the schema of the `people` table:

In [11]:
!psql -h localhost -d baseball -c '\d people';

                                  Table "public.people"
    Column     |          Type          | Collation | Nullable |         Default         
---------------+------------------------+-----------+----------+-------------------------
 player_id     | character varying(10)  |           | not null | NULL::character varying
 birth_year    | integer                |           |          | 
 birth_month   | integer                |           |          | 
 birthday      | integer                |           |          | 
 birth_country | character varying(50)  |           |          | NULL::character varying
 birth_state   | character varying(50)  |           |          | NULL::character varying
 birth_city    | character varying(50)  |           |          | NULL::character varying
 death_year    | integer                |           |          | 
 death_month   | integer                |           |          | 
 death_day     | integer                |           |          | 
 death_count

## A Bit About Baseball

Intricate knowledge of baseball, baseball history, etc. is **not** required for this project. But a basic understanding of the game can be useful. If you'd like, you can check out the [Wikipedia entry for Major League Baseball](https://en.wikipedia.org/wiki/Major_League_Baseball). In practice, we'd encourage you to avoid coming to conclusions nor doing deep analysis with datasets where you don't have the full context, but for this assignment, the details of the sport are not the goal.

You can also read more about the [Baseball Hall of Fame](https://en.wikipedia.org/wiki/List_of_members_of_the_Baseball_Hall_of_Fame). Baseball players who excel in the sport can be inducted into the Hall of Fame.

Baseball, since its inception in the late 19th century, has had an incredible track record of keeping very detailed statistics about even minor aspects of the game. This makes it a fun study for a budding (or well-experienced) data scientist.

**The Briefest of Background:**
The game of baseball is played over 9 innings, between an 'offense' (a batter, and folks running around the bases) and 'defense' (a pitcher, catcher, basemen, and others). The objective, like most sportsball games, is to score the most points -- in this case, by the batter hitting the ball, and running around the bases. 
In each game, both the offense an defense have a chance to bat, where players get an "at bat" (`ab`) in the database. Each 'at bat' results in the player hitting a home run (and scoring), getting "on base" (where they can later try to run to home plate), or getting "out". After the offense gets 3 outs, their turn to score is over until the next inning. (There are many more rules than these... but that's enough to make sense of the data.)

### A Bit About the Database

To be quite honest, the schema of this database isn't our favorite. It is full of abbreivated column names, mixing data types, and seldom enforces primary or foreign key relationships. The horrors!! Nevertheless, real life ain't perfect, so we'll continue to learn how to explore and get comfortable with unknown schemas. A few general tips:

* While foreign keys aren't enforced anything with an `_id` will have consistent values. (i.e. You can treat these like primary or foreign keys on tables.)
* `year_id` is called an "id", but it is merely the calendar year in which an event occured.
* Many of the game related attributes are ruthlessly abbreviated:
    * `ab` is for _at bats_
    * `g_` is for _game..._
    * `hof` is for _hall of fame_
    * Other examples (you don't need to know these, but may come across them):
    * `lg` is for _league_
    * `gp` is games played
    * `1b`, `2b`, `3b`, `hb` refer to 1st, 2nd, 3rd and home base
* In the hall of fame table `inducted` is a string value where `Y` means true
     * ...but of course, you know that this really ought to be a boolean... 

Optionally: There is a [handy guide to an R package](https://cran.r-project.org/web/packages/Lahman/Lahman.pdf) of the Lahman Basebase db. Some of the table and column names are styled slightly differently, but this may provide additional context.

---

## Navigating the Notebook

This notebook is very long! We **highly recommend using the table of contents feature** by clicking on the button with 3 dots and lines on the left sidebar.

<br><br>
<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

# Question 0: PostgreSQL Explain Analyze

<div class="alert alert-block alert-danger">
Please read through this section carefully, as a vast majority of the project will require you to inspect query plans via interpreting the output of the <code>EXPLAIN ANALYZE</code> command.
</div>

Read through the following articles to see how you can interpret the output of `EXPLAIN ANALYZE`:

1. Everything before "Tools to interpret `EXPLAIN ANALYZE` output" in [this article](https://www.cybertec-postgresql.com/en/how-to-interpret-postgresql-explain-analyze-output/)
2. PostgreSQL [documentation 14.1.2](https://www.postgresql.org/docs/current/using-explain.html#USING-EXPLAIN-ANALYZE)

<div class="alert alert-block alert-info">    
<b>Here are some key things to note for all question parts:</b>
<ul>
<li>When we ask you to identify the <b>query cost</b>, we are looking for the <b>total cost</b>.</li>
    <ul>
    <li>There are two cost values: the first is the <b>startup cost</b> (cost to return the first row) and the second is the <b>total cost</b> (cost to return all rows).</li>
    <li>The unit for the estimated query cost is an arbitrary estimation of disk I/O (1 is the cost for reading an 8kB page during a sequential scan).</li>
        <li>Feel free to round the query cost / time to the nearest integer, but we'll accept anything more exact.</li>
    </ul>
<li>When we ask you to identify the <b>query time</b>, we are looking for the <b>execution time</b> (in ms).</li>
    <ul>
        <li>We recognize that the execution time may vary between different cell executions, so the autograder will tolerate a reasonable range.</li>
    </ul>
</ul>
</div>

Now, inspect the query plan above by following the below steps:

1. Run the query below (it's the same one from the screenshot).

In [12]:
%%sql --save query_0 result_0 <<
SELECT *
FROM people AS p
INNER JOIN college_playing AS cp
ON p.player_id = cp.player_id;

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

17350 rows affected.

2. Run the below cell to cache the query and view the first 3 rows.

In [13]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_0 = %sqlcmd snippets query_0
grading_util.save_results("result_0", query_0, result_0);
result_0.DataFrame().head(3)

,player_id,birth_year,birth_month,birthday,birth_country,birth_state,birth_city,death_year,death_month,death_day,...,height,bats,throws,debut,final_game,retro_id,bbref_id,player_id,school_id,year_id
0,aardsda01,1981,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,...,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01,aardsda01,pennst,2001
1,aardsda01,1981,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,...,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01,aardsda01,rice,2002
2,aardsda01,1981,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,...,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01,aardsda01,rice,2003


3. Run the below cell to `EXPLAIN ANALYZE` the saved query. **For this entire project when running `EXPLAIN ANALYZE`, you may ignore the "unsupported syntax" error message if it appears.**

In [14]:
!psql -h localhost -d baseball -c 'EXPLAIN ANALYZE {query_0}'

                                                                           QUERY PLAN                                                                           
----------------------------------------------------------------------------------------------------------------------------------------------------------------
 Merge Join  (cost=0.57..3636.67 rows=17350 width=1788) (actual time=0.056..17.464 rows=17350 loops=1)
   Merge Cond: ((p.player_id)::text = (cp.player_id)::text)
   ->  Index Scan using master_pkey on people p  (cost=0.29..2298.84 rows=19370 width=1698) (actual time=0.019..3.410 rows=19368 loops=1)
   ->  Index Only Scan using college_playing_pkey on college_playing cp  (cost=0.29..1072.54 rows=17350 width=90) (actual time=0.030..4.188 rows=17350 loops=1)
         Heap Fetches: 17350
 Planning Time: 0.548 ms
 Execution Time: 18.106 ms
(7 rows)



4. Finally, record the **cost** and **time** for the sample query. For all questions within this project, we will accept a range of values for your query costs and query timings.

In [15]:
sample_query_cost = 1193.88
sample_query_timing = 15.043

In [16]:
grader.check("q0")

q0 results: All test cases passed!

<br/><br/>
<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

# Question 1: Queries and Views, Part 1

In Questions 1 and 2, you will compare and contrast writing queries with subqueries and views.

## Question 1a
Write a query that finds `people.name_first`, `people.name_last`, `people.player_id` and `hall_of_fame.year_id` of all people who were successfully inducted into the Hall of Fame.

**Note**: Your query should **NOT** use any sub-queries. This is what your table header should look like:

| name_first | name_last | player_id | year_id |
| --- | --- | --- | --- |

In [17]:
%%sql --save query_1a result_1a <<
SELECT p.name_first, p.name_last, p.player_id, h.year_id
FROM people AS p 
JOIN hall_of_fame AS h ON p.player_id = h.player_id
WHERE h.inducted = 'Y';

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

323 rows affected.

In [18]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_1a = %sqlcmd snippets query_1a
grading_util.save_results("result_1a", query_1a, result_1a)
result_1a.DataFrame().head(3)

,name_first,name_last,player_id,year_id
0,Ty,Cobb,cobbty01,1936
1,Babe,Ruth,ruthba01,1936
2,Honus,Wagner,wagneho01,1936


In [19]:
grader.check("q1a")

q1a results: All test cases passed!

<br><br>

---

## Question 1b
In this question, we will compare the query you wrote in Question 1a against the provided query below in Question 1bi by inspecting both query plans.

### Question 1bi
Inspect the query plan for `provided_query` and the query you wrote in Question 1a by running the cells below.

(Note: The semicolon strip in the `EXPLAIN ANALYZE` cell is to avoid executing an empty query with double-semicolons, which causes an error).

In [20]:
%%sql --save provided_query provided_result <<
-- just run this cell
SELECT name_first, name_last, p.player_id, year_id
FROM people AS p,
(
  SELECT * FROM hall_of_fame WHERE inducted = 'Y'
) AS hof
WHERE p.player_id = hof.player_id;

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

323 rows affected.

In [21]:
# just run this cell
provided_query = %sqlcmd snippets provided_query
!psql -h localhost -d baseball -c "EXPLAIN ANALYZE {provided_query}"

                                                           QUERY PLAN                                                           
--------------------------------------------------------------------------------------------------------------------------------
 Nested Loop  (cost=0.29..262.79 rows=21 width=278) (actual time=0.060..2.091 rows=323 loops=1)
   ->  Seq Scan on hall_of_fame  (cost=0.00..96.39 rows=21 width=42) (actual time=0.020..0.538 rows=323 loops=1)
         Filter: ((inducted)::text = 'Y'::text)
         Rows Removed by Filter: 3868
   ->  Index Scan using master_pkey on people p  (cost=0.29..7.92 rows=1 width=274) (actual time=0.004..0.004 rows=1 loops=323)
         Index Cond: ((player_id)::text = (hall_of_fame.player_id)::text)
 Planning Time: 0.675 ms
 Execution Time: 2.158 ms
(8 rows)



In [22]:
# just run this cell
!psql -h localhost -d baseball -c "EXPLAIN ANALYZE {query_1a}"

                                                           QUERY PLAN                                                           
--------------------------------------------------------------------------------------------------------------------------------
 Nested Loop  (cost=0.29..262.79 rows=21 width=278) (actual time=0.067..2.363 rows=323 loops=1)
   ->  Seq Scan on hall_of_fame h  (cost=0.00..96.39 rows=21 width=42) (actual time=0.024..0.626 rows=323 loops=1)
         Filter: ((inducted)::text = 'Y'::text)
         Rows Removed by Filter: 3868
   ->  Index Scan using master_pkey on people p  (cost=0.29..7.92 rows=1 width=274) (actual time=0.005..0.005 rows=1 loops=323)
         Index Cond: ((player_id)::text = (h.player_id)::text)
 Planning Time: 0.610 ms
 Execution Time: 2.434 ms
(8 rows)



Record the **execution time** and **cost** for each query.

In [23]:
provided_query_cost = 959.06
provided_query_timing = 9.737
your_query_cost = 959.06
your_query_timing = 8.703

In [24]:
grader.check("q1bi")

q1bi results: All test cases passed!


### Question 1bii

Given your findings from inspecting the query plans of the two queries, consider the following statements.

**Assign the variable `q1bii` to a list of _ALL_ of the below statements that are true.**


Consider the following statements:
<br>
A. Both the queries have the same cost.
<br>
B. The provided query has a faster execution time because it makes use of a subquery.
<br>
C. The query you wrote has a faster execution time because it does not make use a subquery.
<br>
D. The provided query has less cost because it makes use of a subquery.
<br>
E. The query you wrote has less cost because it does not make use a subquery.
<br>
F. The queries have the same output.
<br>
G. The queries do not have the same output.
    
**Note:** Your answer should have the format like this if you think A and B are both true: `q1bii = ['A', 'B']`. The autograder is **case sensitive** but order should not matter.

In [25]:
q1bii = ['A','F']

In [26]:
grader.check("q1bii")

q1bii results: All test cases passed!

<br/><br/><br/>

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />


# Question 2: Queries and Views, Part 2

In this question, you will continue analyzing queries with/without views and materialized views.

<br/><br/>

---

## Question 2a

Write a query that returns the people who were successfully inducted into the Hall of Fame and played in college at a school located in California. For each player, return their `name_first`, `name_last`, `player_id`, `school_id`, and `year_id`. Order by the `year_id` (ascending) and break ties on `player_id` (ascending). This is what your table header should look like:

| name_first | name_last | player_id | school_id | year_id |
| --- | --- | --- | --- | --- |

**Note**: 
- **Do NOT use any views, materialized views, CTEs, or subqueries**
- For this query, `year_id` refers to player's year of induction into the Hall of Fame.
- For the baseball fanatics, FYI this dataset does not include *all* hall of fame inductees in existence and may be missing some. However, this should not affect your result; please query from the tables as given.

In [27]:
%%sql --save query_2a result_2a <<
SELECT p.name_first, p.name_last, p.player_id, cp.school_id, hof.year_id
FROM people p
JOIN college_playing cp ON p.player_id = cp.player_id
JOIN hall_of_fame hof ON p.player_id = hof.player_id
JOIN schools s ON cp.school_id = s.school_id
WHERE hof.inducted = 'Y'
  AND s.school_state = 'CA'
ORDER BY hof.year_id ASC, p.player_id ASC;

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

23 rows affected.

In [28]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_2a = %sqlcmd snippets query_2a
grading_util.save_results("result_2a", query_2a, result_2a)
result_2a

name_first,name_last,player_id,school_id,year_id
Jackie,Robinson,robinja02,ucla,1962
Harry,Hooper,hoopeha01,stmarysca,1971
Joe,Morgan,morgajo02,camerri,1990
Tom,Seaver,seaveto01,cafrecc,1992
Tom,Seaver,seaveto01,usc,1992
Ozzie,Smith,smithoz01,calpoly,2002
Ozzie,Smith,smithoz01,calpoly,2002
Ozzie,Smith,smithoz01,calpoly,2002
Ozzie,Smith,smithoz01,calpoly,2002
Tony,Gwynn,gwynnto01,sandiegost,2007


In [29]:
grader.check("q2a")

q2a results: All test cases passed!

<br/><br/>

---

## Question 2 Tutorial

We are now going to use the query you wrote in the previous part to generate a view, called `inducted_hof_ca`, and a materialized view, `inducted_hof_ca_mat`.

Run the below cells. You do not need to do anything more for this part. 

In [30]:
%%sql
/* just run this cell */
DROP VIEW IF EXISTS inducted_hof_ca;
CREATE VIEW inducted_hof_ca AS {{query_2a.strip(';')}};

SELECT * FROM inducted_hof_ca;

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

23 rows affected.

name_first,name_last,player_id,school_id,year_id
Jackie,Robinson,robinja02,ucla,1962
Harry,Hooper,hoopeha01,stmarysca,1971
Joe,Morgan,morgajo02,camerri,1990
Tom,Seaver,seaveto01,cafrecc,1992
Tom,Seaver,seaveto01,usc,1992
Ozzie,Smith,smithoz01,calpoly,2002
Ozzie,Smith,smithoz01,calpoly,2002
Ozzie,Smith,smithoz01,calpoly,2002
Ozzie,Smith,smithoz01,calpoly,2002
Tony,Gwynn,gwynnto01,sandiegost,2007


In [31]:
%%sql
/* just run this cell */
DROP MATERIALIZED VIEW IF EXISTS inducted_hof_ca_mat;
CREATE MATERIALIZED VIEW inducted_hof_ca_mat AS {{query_2a.strip(';')}};

SELECT * FROM inducted_hof_ca_mat;

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

23 rows affected.

23 rows affected.

name_first,name_last,player_id,school_id,year_id
Jackie,Robinson,robinja02,ucla,1962
Harry,Hooper,hoopeha01,stmarysca,1971
Joe,Morgan,morgajo02,camerri,1990
Tom,Seaver,seaveto01,cafrecc,1992
Tom,Seaver,seaveto01,usc,1992
Ozzie,Smith,smithoz01,calpoly,2002
Ozzie,Smith,smithoz01,calpoly,2002
Ozzie,Smith,smithoz01,calpoly,2002
Ozzie,Smith,smithoz01,calpoly,2002
Tony,Gwynn,gwynnto01,sandiegost,2007


<br/><br/>

---
## Question 2b

For this question, we want to compute the count of players who were inducted into the Hall of Fame and played baseball at a college in California for each `school_id` and `year_id` combination ordered by ascending `year_id`.

You should write three queries that accomplish this task, but with different strategies:
* Question 2bi: Use the `inducted_hof_ca` view;
* Question 2bii Use the `inducted_hof_ca_mat` view; and
* Question 2biii: **Do not use `inducted_hof_ca` view, `inducted_hof_ca_mat` materialized view, any common table expressions (CTEs), nor any subqueries.**

For all subparts in Q2b, your table header should look like this:

| school_id | year_id | count |
| --- | --- | ---|

### Question 2bi

Write a query to accomplish the task above using the `inducted_hof_ca` view.

In [32]:
%%sql --save query_2bi result_2bi <<
SELECT school_id, year_id, COUNT(player_id) AS count
FROM inducted_hof_ca
GROUP BY school_id, year_id
ORDER BY year_id ASC, school_id ASC;

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

13 rows affected.

In [33]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_2bi = %sqlcmd snippets query_2bi
grading_util.save_results("result_2bi", query_2bi, result_2bi)
result_2bi

school_id,year_id,count
ucla,1962,1
stmarysca,1971,1
camerri,1990,1
cafrecc,1992,1
usc,1992,1
calpoly,2002,4
sandiegost,2007,3
capasad,2008,1
sandiegost,2010,2
calavco,2011,1


In [34]:
grader.check("q2bi")

q2bi results: All test cases passed!

<br/><br/>

### Question 2bii

Now, write the query a second time using the materialized view `inducted_hof_ca_mat`.

In [35]:
%%sql --save query_2bii result_2bii <<
SELECT school_id, year_id, COUNT(player_id) AS count
FROM inducted_hof_ca_mat
GROUP BY school_id, year_id
ORDER BY year_id ASC, school_id ASC;

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

13 rows affected.

In [36]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_2bii = %sqlcmd snippets query_2bii
grading_util.save_results("result_2bii", query_2bii, result_2bii)
result_2bii

school_id,year_id,count
ucla,1962,1
stmarysca,1971,1
camerri,1990,1
cafrecc,1992,1
usc,1992,1
calpoly,2002,4
sandiegost,2007,3
capasad,2008,1
sandiegost,2010,2
calavco,2011,1


In [37]:
grader.check("q2bii")

q2bii results: All test cases passed!

<br/><br/>

### Question 2biii

Finally, write the query a third time. Do **NOT** use the `inducted_hof_ca` view, nor the `inducted_hof_ca_mat` materialized view, nor any common table expressions (CTEs), nor any subqueries.

In [38]:
%%sql --save query_2biii result_2biii <<
SELECT s.school_id, h.year_id, COUNT(p.player_id) AS count
FROM hall_of_fame h
JOIN college_playing c ON h.player_id = c.player_id
JOIN people p ON p.player_id = h.player_id
JOIN schools s ON s.school_id = c.school_id
WHERE h.inducted = 'Y'
  AND s.school_state = 'CA'
GROUP BY s.school_id, h.year_id
ORDER BY h.year_id ASC, s.school_id ASC;

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

13 rows affected.

In [39]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_2biii = %sqlcmd snippets query_2biii
grading_util.save_results("result_2biii", query_2biii, result_2biii)
result_2biii

school_id,year_id,count
ucla,1962,1
stmarysca,1971,1
camerri,1990,1
cafrecc,1992,1
usc,1992,1
calpoly,2002,4
sandiegost,2007,3
capasad,2008,1
sandiegost,2010,2
calavco,2011,1


In [40]:
grader.check("q2biii")

q2biii results: All test cases passed!

<br/><br/>

---

## Question 2c
Inspect the query plans for the three queries you wrote above by running the following cells.

In [41]:
# just run this cell
!psql -h localhost -d baseball -c "EXPLAIN ANALYZE {query_2bi}";

                                                                                      QUERY PLAN                                                                                      
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=325.73..325.75 rows=1 width=60) (actual time=4.068..4.076 rows=13 loops=1)
   Group Key: inducted_hof_ca.year_id, inducted_hof_ca.school_id
   ->  Sort  (cost=325.73..325.74 rows=1 width=90) (actual time=4.063..4.065 rows=23 loops=1)
         Sort Key: inducted_hof_ca.year_id, inducted_hof_ca.school_id
         Sort Method: quicksort  Memory: 26kB
         ->  Subquery Scan on inducted_hof_ca  (cost=325.71..325.72 rows=1 width=90) (actual time=4.013..4.018 rows=23 loops=1)
               ->  Sort  (cost=325.71..325.71 rows=1 width=326) (actual time=4.012..4.014 rows=23 loops=1)
                     Sort 

In [42]:
# just run this cell
!psql -h localhost -d baseball -c "EXPLAIN ANALYZE {query_2bii}";

                                                          QUERY PLAN                                                          
------------------------------------------------------------------------------------------------------------------------------
 Sort  (cost=23.67..24.17 rows=200 width=60) (actual time=0.054..0.055 rows=13 loops=1)
   Sort Key: year_id, school_id
   Sort Method: quicksort  Memory: 25kB
   ->  HashAggregate  (cost=14.03..16.02 rows=200 width=60) (actual time=0.018..0.021 rows=13 loops=1)
         Group Key: year_id, school_id
         Batches: 1  Memory Usage: 40kB
         ->  Seq Scan on inducted_hof_ca_mat  (cost=0.00..12.30 rows=230 width=90) (actual time=0.003..0.005 rows=23 loops=1)
 Planning Time: 0.261 ms
 Execution Time: 0.152 ms
(9 rows)



In [43]:
# just run this cell
!psql -h localhost -d baseball -c "EXPLAIN ANALYZE {query_2biii}";

                                                                               QUERY PLAN                                                                                
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=357.63..357.81 rows=9 width=60) (actual time=3.587..3.596 rows=13 loops=1)
   Group Key: h.year_id, s.school_id
   ->  Sort  (cost=357.63..357.65 rows=9 width=90) (actual time=3.582..3.585 rows=23 loops=1)
         Sort Key: h.year_id, s.school_id
         Sort Method: quicksort  Memory: 26kB
         ->  Hash Join  (cost=28.74..357.49 rows=9 width=90) (actual time=1.496..3.544 rows=23 loops=1)
               Hash Cond: ((c.school_id)::text = (s.school_id)::text)
               ->  Nested Loop  (cost=0.57..324.52 rows=1822 width=90) (actual time=0.141..3.314 rows=137 loops=1)
                     Join Filter: ((h.player_id)::text

Then, record the execution time and cost for each query.

In [44]:
with_view_cost = 519.49
with_view_timing = 6.06
with_materialized_view_cost = 24.17
with_materialized_view_timing = 0.059
without_view_cost = 521.03
without_view_timing = 4.539

In [45]:
grader.check("q2c")

q2c results: All test cases passed!

<br/><br/>

---

## Question 2d

Given your findings from inspecting the query plans in this question, as well as your understanding of views and materialized views from lectures, discuss the takeaways of using views and materialized views.

### Question 2di

**Assign the variable `q2di` to a list of _ALL_ of the below statements that are true.**

A. Views will reduce the execution time and the cost of a query.<br/>
B. Views will reduce the execution time of a query, but not the cost.<br/>
C. Views will reduce the cost of a query, but not the execution time.<br/>
D. Materialized views reduce the execution time and the cost of a query.<br/>
E. Materialized views reduce the execution time, but not cost of a query.<br/>
F. Materialized views reduce the cost of a query, but not the execution time.<br/>
G. Materialized views will result in the same query plan as a query using views.<br/>
H. Materialized views and views take the same time to create.<br/>
I. Materialized views take less time to create than a view.<br/>
J. Materialized views take more time to create than a view.<br/>
    
**Note:** Your answer should have the format like this if you think A and B are both true: `q2di = ['A', 'B']`. The autograder is **case sensitive** but order should not matter.

In [46]:
q2di = ['D','J']

In [47]:
grader.check("q2di")

q2di results: All test cases passed!

<!-- BEGIN QUESTION -->

### Question 2dii

1. **Explain your answer** to the previous part (Question 2di) based on your knowledge from lectures and details from the query plans.
2. If there were any options you did NOT select, **choose any one of them and explain why you did not select it.**

**Please answer in at most 5 sentences** and **explicitly state which answer option(s) you chose or didn't choose** in addition to your explanations. For example, you could write, "I chose (A) because..." or "I did NOT choose (B) because..."

I chose (D) because materialized views provide already generated results that can significantly reduce both the execution time and cost of queries by eliminating the need to recompute the data each time. I also chose (J) because creating materialized views typically requires more time than creating regular views, as materialized views store the actual data rather than just the SQL query. I did NOT choose (A) or (B) because views do not inherently reduce execution time or cost, they simply provide a convenient way to contain complex queries without improving performance.

<!-- END QUESTION -->

<br><br>

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

# Question 3: Predicate Pushdown
In this question, we will explore the impact of predicates (i.e., filters) on a query's execution, particularly inspecting when the optimizer applies predicates.


## Question 3a

Recall the `inducted_hof_ca` view created in `Question 2`. Run `EXPLAIN ANALYZE` for a query that that gets all rows from the view, and record the execution time and cost.

In [48]:
%%sql
EXPLAIN ANALYZE SELECT * FROM inducted_hof_ca;

'EXPLAIN ANALYZE SELECT * FROM inducted_hof_ca' contains unsupported syntax. Falling back to parsing as a 'Command'.


Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

20 rows affected.

QUERY PLAN
Sort (cost=325.71..325.71 rows=1 width=326) (actual time=3.411..3.414 rows=23 loops=1)
"Sort Key: hof.year_id, p.player_id"
Sort Method: quicksort Memory: 26kB
-> Nested Loop (cost=0.85..325.70 rows=1 width=326) (actual time=1.189..3.395 rows=23 loops=1)
-> Nested Loop (cost=0.57..319.95 rows=19 width=326) (actual time=0.104..3.041 rows=137 loops=1)
-> Nested Loop (cost=0.29..262.79 rows=21 width=316) (actual time=0.049..1.913 rows=323 loops=1)
-> Seq Scan on hall_of_fame hof (cost=0.00..96.39 rows=21 width=42) (actual time=0.014..0.544 rows=323 loops=1)
Filter: ((inducted)::text = 'Y'::text)
Rows Removed by Filter: 3868
-> Index Scan using master_pkey on people p (cost=0.29..7.92 rows=1 width=274) (actual time=0.004..0.004 rows=1 loops=323)


In [49]:
query_view_cost = 527.10
query_view_timing = 4.199

In [50]:
grader.check("q3a")

q3a results: All test cases passed!

<br><br>

---

## Question 3b

Now, run `EXPLAIN ANALYZE` on the same query as 3a, except now add a filter to only return rows from `inducted_hof_ca` where the year is after 2010. Inspect the query plan and record the execution time and cost.

In [51]:
%%sql
EXPLAIN ANALYZE SELECT * FROM inducted_hof_ca 
WHERE year_id > 2010;

'EXPLAIN ANALYZE SELECT * FROM inducted_hof_ca 
WHERE year_id > 2010' contains unsupported syntax. Falling back to parsing as a 'Command'.


Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

20 rows affected.

QUERY PLAN
Sort (cost=185.88..185.89 rows=1 width=326) (actual time=1.134..1.136 rows=8 loops=1)
"Sort Key: hof.year_id, p.player_id"
Sort Method: quicksort Memory: 25kB
-> Nested Loop (cost=0.85..185.87 rows=1 width=326) (actual time=0.530..1.119 rows=8 loops=1)
-> Nested Loop (cost=0.57..184.05 rows=6 width=326) (actual time=0.517..0.992 rows=27 loops=1)
-> Nested Loop (cost=0.29..165.00 rows=7 width=316) (actual time=0.467..0.779 rows=31 loops=1)
-> Seq Scan on hall_of_fame hof (cost=0.00..106.86 rows=7 width=42) (actual time=0.435..0.521 rows=31 loops=1)
Filter: ((year_id > 2010) AND ((inducted)::text = 'Y'::text))
Rows Removed by Filter: 4160
-> Index Scan using master_pkey on people p (cost=0.29..8.30 rows=1 width=274) (actual time=0.007..0.007 rows=1 loops=31)


In [52]:
query_view_with_filter_cost = 210
query_view_with_filter_timing = 0.7

In [53]:
grader.check("q3b")

q3b results: All test cases passed!

<!-- BEGIN QUESTION -->

## Question 3c

Given your findings from inspecting the query plans of queries from Questions 3a and 3b, fill in the blank and **justify your answer**. Explain your answer based on your knowledge from lectures, and details from the query plans. Your response should be no longer than 3 sentences.

**Note:** Your answer should be formatted as follows if you think option A is true: `(A) because ...`

**Adding a filter ___ the cost.**
<br>
A. increased
<br>
B. decreased
<br>
C. did not change

(B) because the query plan for the filtered query (3b) shows a cost of 210, which is lower than the cost for the unfiltered query (3a), indicating that fewer resources are required to retrieve the relevant data when a filter is applied.

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

<br><br>

---

## Question 3d

Given your findings from inspecting the query plans of queries from Questions 3a and 3b, fill in the blank and **justify your answer**. Explain your answer based on your knowledge from lectures, and details from the query plans. Your response should be no longer than 3 sentences.

**Note:** Your answer should be formatted as follows if you think option A is true: `(A) because ...`

**Adding a filter ___ the execution time.**
<br>
A. increased
<br>
B. decreased
<br>
C. did not change

(B) because the execution time for the filtered query (3b) 1.1 is significantly lower than the execution time for the unfiltered query 4.4 (3a), which basically demonstrates that the filter optimizes the query performance by reducing the number of rows processed.

<!-- END QUESTION -->

<br><br>

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

# Question 4: Join Approaches

In this question, we'll explore different join approaches (Nested Loop Join, Merge Join, Hash Join) and discuss how the query optimizer picks the best approach.

<br/><br/>

---

## Question 4a
Perform an inner join on the `people` and `college_playing` tables on the `player_id` column. Select all columns. (No need to run `EXPLAIN ANALYZE` here, we do it for you below.)

In [54]:
%%sql --save query_4a result_4a <<
SELECT *
FROM people p
INNER JOIN college_playing cp ON p.player_id = cp.player_id;

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

17350 rows affected.

In [55]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_4a = %sqlcmd snippets query_4a
grading_util.save_results("result_4a", query_4a, result_4a);

display(result_4a.DataFrame().head(3))

!psql -h localhost -d baseball -c "EXPLAIN ANALYZE {query_4a}";

,player_id,birth_year,birth_month,birthday,birth_country,birth_state,birth_city,death_year,death_month,death_day,...,height,bats,throws,debut,final_game,retro_id,bbref_id,player_id,school_id,year_id
0,aardsda01,1981,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,...,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01,aardsda01,pennst,2001
1,aardsda01,1981,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,...,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01,aardsda01,rice,2002
2,aardsda01,1981,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,...,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01,aardsda01,rice,2003


                                                                           QUERY PLAN                                                                           
----------------------------------------------------------------------------------------------------------------------------------------------------------------
 Merge Join  (cost=0.57..3636.67 rows=17350 width=1788) (actual time=0.056..16.816 rows=17350 loops=1)
   Merge Cond: ((p.player_id)::text = (cp.player_id)::text)
   ->  Index Scan using master_pkey on people p  (cost=0.29..2298.84 rows=19370 width=1698) (actual time=0.017..3.260 rows=19368 loops=1)
   ->  Index Only Scan using college_playing_pkey on college_playing cp  (cost=0.29..1072.54 rows=17350 width=90) (actual time=0.032..4.020 rows=17350 loops=1)
         Heap Fetches: 17350
 Planning Time: 0.577 ms
 Execution Time: 17.386 ms
(7 rows)



If you haven't already, run the cell above to inspect the query plan for your command.

**Which join approach did the query optimizer choose?** 

A. Nested Loop Join<br/>
B. Merge Join<br/>
C. Hash Join<br/>
D. None of the Above

Assign the variable `q4a` to the correct letter choice above, e.g., `q4a = 'A'`.

In [56]:
q4a = 'C'

In [57]:
grader.check("q4a")

q4a results: All test cases passed!

<br><br>

---

## Question 4b

Perform the same query in Question 4a, but now also **sort the output by `player_id`**.

In [58]:
%%sql --save query_4b result_4b <<
SELECT *
FROM people p
INNER JOIN college_playing cp ON p.player_id = cp.player_id
ORDER BY p.player_id
;

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

17350 rows affected.

In [59]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_4b = %sqlcmd snippets query_4b
grading_util.save_results("result_4b", query_4b, result_4b);

display(result_4b.DataFrame().head(3))

!psql -h localhost -d baseball -c "EXPLAIN ANALYZE {query_4b}";

,player_id,birth_year,birth_month,birthday,birth_country,birth_state,birth_city,death_year,death_month,death_day,...,height,bats,throws,debut,final_game,retro_id,bbref_id,player_id,school_id,year_id
0,aardsda01,1981,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,...,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01,aardsda01,pennst,2001
1,aardsda01,1981,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,...,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01,aardsda01,rice,2002
2,aardsda01,1981,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,...,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01,aardsda01,rice,2003


                                                                           QUERY PLAN                                                                           
----------------------------------------------------------------------------------------------------------------------------------------------------------------
 Merge Join  (cost=0.57..3636.67 rows=17350 width=1788) (actual time=0.058..17.111 rows=17350 loops=1)
   Merge Cond: ((p.player_id)::text = (cp.player_id)::text)
   ->  Index Scan using master_pkey on people p  (cost=0.29..2298.84 rows=19370 width=1698) (actual time=0.016..3.329 rows=19368 loops=1)
   ->  Index Only Scan using college_playing_pkey on college_playing cp  (cost=0.29..1072.54 rows=17350 width=90) (actual time=0.034..4.060 rows=17350 loops=1)
         Heap Fetches: 17350
 Planning Time: 0.542 ms
 Execution Time: 17.699 ms
(7 rows)



If you haven't already, run the cell above to inspect the query plan for your command.

**Which join approach did the query optimizer choose?** 

A. Nested Loop Join<br/>
B. Merge Join<br/>
C. Hash Join<br/>
D. None of the Above

Assign the variable `q4b` to the correct letter choice above, e.g., `q4b = 'A'`.

In [60]:
q4b = 'B'

In [61]:
grader.check("q4b")

q4b results: All test cases passed!

<br><br>

---
## Question 4c
Write a query to retrieve all possible player pair permutations (e.g. `(Player 1, Player 2)` is treated as a different pairing from `(Player 2, Player 1)` because the order matters). Select all columns, but **limit to 1000 rows** to ensure your query doesn't take an exorbitant amount of time to run.

**Hint:** You can do this by performing an inner join of the `people` table on itself with an inequality condition.

In [62]:
%%sql --save query_4c result_4c <<
SELECT p1.*, p2.*
FROM people p1
JOIN people p2 ON p1.player_id <> p2.player_id
LIMIT 1000
;

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

1000 rows affected.

In [63]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_4c = %sqlcmd snippets query_4c
grading_util.save_results("result_4c", query_4c, result_4c);

display(result_4c.DataFrame().head(3))

!psql -h localhost -d baseball -c "EXPLAIN ANALYZE {query_4c}";

,player_id,birth_year,birth_month,birthday,birth_country,birth_state,birth_city,death_year,death_month,death_day,...,name_last,name_given,weight,height,bats,throws,debut,final_game,retro_id,bbref_id
0,aardsda01,1981,12,27,USA,CO,Denver,None,None,None,...,Aaron,Henry Louis,180.0,72.0,R,R,1954-04-13,1976-10-03,aaroh101,aaronha01
1,aardsda01,1981,12,27,USA,CO,Denver,None,None,None,...,Aaron,Tommie Lee,190.0,75.0,R,R,1962-04-10,1971-09-26,aarot101,aaronto01
2,aardsda01,1981,12,27,USA,CO,Denver,None,None,None,...,Aase,Donald William,190.0,75.0,R,R,1977-07-26,1990-10-03,aased001,aasedo01


                                                        QUERY PLAN                                                         
---------------------------------------------------------------------------------------------------------------------------
 Limit  (cost=0.00..44.50 rows=1000 width=3396) (actual time=0.021..0.528 rows=1000 loops=1)
   ->  Nested Loop  (cost=0.00..16694169.95 rows=375177530 width=3396) (actual time=0.020..0.468 rows=1000 loops=1)
         Join Filter: ((p1.player_id)::text <> (p2.player_id)::text)
         Rows Removed by Join Filter: 1
         ->  Seq Scan on people p1  (cost=0.00..619.70 rows=19370 width=1698) (actual time=0.013..0.013 rows=1 loops=1)
         ->  Seq Scan on people p2  (cost=0.00..619.70 rows=19370 width=1698) (actual time=0.001..0.125 rows=1001 loops=1)
 Planning Time: 0.540 ms
 Execution Time: 0.612 ms
(8 rows)



If you haven't already, run the cell above to inspect the query plan for your command.

**Which join approach did the query optimizer choose?** 

A. Nested Loop Join<br/>
B. Merge Join<br/>
C. Hash Join<br/>
D. None of the Above

Assign the variable `q4c` to the correct letter choice above, e.g., `q4c = 'A'`.

In [64]:
q4c = 'A'

In [65]:
grader.check("q4c")

q4c results: All test cases passed!

<!-- BEGIN QUESTION -->

<br><br>

---
## Question 4d

Given your findings above, why did the query optimizer ultimately choose the specific join approach you found in each of the above three scenarios in Questions 4a, 4b, and 4c? Feel free to discuss the pros and cons of each join approach as well.

If you feel stuck, here are some things to consider: Does a non-equijoin constrain us to certain join approaches? What's an added benefit in regards to the output of merge join? How does table size affect which join is used?

**Note:** Restate your answer for each of the subparts. You should write no more than 5 sentences. Your answer should be formatted as follows: 
```
Q4a: (A) because ...
Q4b: (A) because ...
Q4c: (A) because ...
```

Q4a: (C) because the optimizer chose a Hash Join, which is effective when both tables are large and there's no sorting requirement for the join. For 4a, the hash join utilized memory to build a hash table for one of the tables i.e (people), allowing for efficient lookups during the join operation.

Q4b: (B) because the optimizer chose a Merge Join, which is optimal when both inputs are sorted on the join keys. Since both tables have indexes on player_id, the merge join efficiently scans through the sorted data, producing sorted output without additional sorting costs. 

Q4c: (A) because the optimizer selected a Nested Loop Join, which is suitable for scenarios involving a bigger number of potential combinations, especially with an inequality condition. Although typically this approach can be inefficient for very large datasets, BUT it was pretty effective here due to the smaller result set limited to 1000 rows. Also, something to note is that The nested loop join doesn't require sorted data, making it flexible for non-equi joins.

<!-- END QUESTION -->

<br/><br/><br/>

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

# Question 5: Indexes, Part 1

In Questions 5, 6, and 7, you will analyze how indexes impact query performance.

<br/>

---

## Question 5a

For each `player_id`, determine their average yearly `salary` (call this `average_salary`), the first year in which they played baseball professionally (call this `first_year`), the last year in which they played baseball professionally (call this `last_year`), and the total number of years they played baseball professionally (call this `num_years`).

Only include players that have batted in exactly 10 games (the number of games in which a player batted can be found in the `g_batting` column of the `appearances` table).

Your query should join the `salaries` and `appearances` table on all the common columns `year_id`, `team_id`, and `player_id`, so feel free to use a natural join. Your table header should look like this:

| player_id | average_salary | first_year | last_year | num_years |
| --- | --- | --- | --- | --- |

In [66]:
%%sql --save query_5a result_5a <<
SELECT a.player_id, AVG(s.salary) AS average_salary, MIN(a.year_id) AS first_year, MAX(a.year_id) AS last_year, COUNT(DISTINCT a.year_id) AS num_years
FROM salaries s
JOIN appearances a ON s.year_id = a.year_id AND s.team_id = a.team_id AND s.player_id = a.player_id
WHERE a.g_batting = 10
GROUP BY a.player_id;

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

134 rows affected.

In [67]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_5a = %sqlcmd snippets query_5a
grading_util.save_results("result_5a", query_5a, result_5a);

display(result_5a.DataFrame().head(3))

!psql -h localhost -d baseball -c "EXPLAIN ANALYZE {query_5a}";

,player_id,average_salary,first_year,last_year,num_years
0,anderla02,240000.0,1986,1986,1
1,ashbyan01,109000.0,1992,1992,1
2,ayraubo01,125000.0,1993,1993,1


                                                                   QUERY PLAN                                                                   
------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=3549.73..3549.78 rows=2 width=62) (actual time=17.329..17.452 rows=134 loops=1)
   Group Key: a.player_id
   ->  Sort  (cost=3549.73..3549.73 rows=2 width=50) (actual time=17.245..17.253 rows=138 loops=1)
         Sort Key: a.player_id
         Sort Method: quicksort  Memory: 35kB
         ->  Hash Join  (cost=2882.32..3549.72 rows=2 width=50) (actual time=11.156..17.099 rows=138 loops=1)
               Hash Cond: ((s.year_id = a.year_id) AND ((s.team_id)::text = (a.team_id)::text) AND ((s.player_id)::text = (a.player_id)::text))
               ->  Seq Scan on salaries s  (cost=0.00..459.28 rows=26428 width=66) (actual time=0.011..1.965 rows=26428 loops=1)
               ->  Has

Inspect the query plan above and record the execution time and cost.

In [68]:
result_5a_cost = 3563.81
result_5a_timing = 17.502

In [69]:
grader.check("q5a")

q5a results: All test cases passed!

<br><br>

---
## Question 5b

Add an index with name `appearances_g_batting_idx` on the `g_batting` column of the `appearances` table.

In [70]:
%%sql
DROP INDEX IF EXISTS appearances_g_batting_idx;
CREATE INDEX appearances_g_batting_idx ON appearances (g_batting)
;

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

++
||
++
++

Now, re-inspect the query plan of the query from `Question 5a` and record its execution time and cost.

In [71]:
# just run this cell
!psql -h localhost -d baseball -c "EXPLAIN ANALYZE {query_5a}";

                                                                           QUERY PLAN                                                                            
-----------------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=2322.41..2322.44 rows=1 width=33) (actual time=8.472..8.600 rows=134 loops=1)
   Group Key: a.player_id
   ->  Sort  (cost=2322.41..2322.41 rows=1 width=21) (actual time=8.406..8.415 rows=138 loops=1)
         Sort Key: a.player_id
         Sort Method: quicksort  Memory: 35kB
         ->  Hash Join  (cost=1655.00..2322.40 rows=1 width=21) (actual time=2.416..8.258 rows=138 loops=1)
               Hash Cond: ((s.year_id = a.year_id) AND ((s.team_id)::text = (a.team_id)::text) AND ((s.player_id)::text = (a.player_id)::text))
               ->  Seq Scan on salaries s  (cost=0.00..459.28 rows=26428 width=25) (actual time=0.011..1.660 rows=26428 loo

In [72]:
result_5b_cost = 2299.44
result_5b_timing = 8.731

In [73]:
grader.check("q5b")

q5b results: All test cases passed!

<br><br>

---
## Question 5c

Write a query to add an index with name `salary_idx` on the `salary` column of the `salaries` table. Make sure to drop the previous index in Question 5b first (which we've provided in the starter code)!

In [74]:
%%sql
DROP INDEX IF EXISTS appearances_g_batting_idx;
DROP INDEX IF EXISTS salary_idx;

CREATE INDEX salary_idx ON salaries(salary);

SELECT 
    a.player_id,
    AVG(s.salary) AS average_salary,
    MIN(a.year_id) AS first_year,
    MAX(a.year_id) AS last_year,
    COUNT(DISTINCT a.year_id) AS num_years
FROM 
    salaries s
JOIN 
    appearances a ON s.year_id = a.year_id AND s.team_id = a.team_id AND s.player_id = a.player_id
WHERE 
    a.g_batting = 10
GROUP BY 
    a.player_id;

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

134 rows affected.

player_id,average_salary,first_year,last_year,num_years
anderla02,240000.0,1986,1986,1
ashbyan01,109000.0,1992,1992,1
ayraubo01,125000.0,1993,1993,1
bankswi01,240000.0,1995,1995,1
batchri01,157500.0,1997,1997,1
batiski01,100000.0,1991,1991,1
batismi01,153000.0,1997,1997,1
bautijo01,150000.0,1997,1997,1
beller01,145000.0,1993,1993,1
benesan01,68000.0,1989,1989,1


Now, re-inspect the query plan of the query from Question 5a and record its execution time and cost.

In [75]:
# just run this cell
%sql EXPLAIN ANALYZE {{query_5a}}

'EXPLAIN ANALYZE SELECT a.player_id, AVG(s.salary) AS average_salary, MIN(a.year_id) AS first_year, M' contains unsupported syntax. Falling back to parsing as a 'Command'.


Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

15 rows affected.

QUERY PLAN
GroupAggregate (cost=3565.23..3565.26 rows=1 width=33) (actual time=15.301..15.430 rows=134 loops=1)
Group Key: a.player_id
-> Sort (cost=3565.23..3565.24 rows=1 width=21) (actual time=15.285..15.293 rows=138 loops=1)
Sort Key: a.player_id
Sort Method: quicksort Memory: 35kB
-> Hash Join (cost=2897.82..3565.22 rows=1 width=21) (actual time=9.352..15.173 rows=138 loops=1)
Hash Cond: ((s.year_id = a.year_id) AND ((s.team_id)::text = (a.team_id)::text) AND ((s.player_id)::text = (a.player_id)::text))
-> Seq Scan on salaries s (cost=0.00..459.28 rows=26428 width=25) (actual time=0.009..1.580 rows=26428 loops=1)
-> Hash (cost=2873.20..2873.20 rows=1407 width=17) (actual time=9.197..9.198 rows=1347 loops=1)
Buckets: 2048 Batches: 1 Memory Usage: 85kB


In [76]:
result_5c_cost = 3563.80
result_5c_timing = 15.065

In [77]:
grader.check("q5c")

q5c results: All test cases passed!

<br><br>

---

## Question 5d

Given your findings from inspecting the query plans with no indexes (Question 5a), an index on `g_batting` (Question 5b), and an index on `salary` (Question 5c), assign the variable `q5d` to a list of **ALL** of the below statements that are true.

A. Adding the `appearances_g_batting` index did not have a significant impact on the query execution time and cost.<br/>
B. Adding the `appearances_g_batting` index did have a significant impact on the query execution time, but not the cost.<br/>
C. Adding the `appearances_g_batting` index did have a significant impact on the query cost, but not the execution time.<br/>
D. Adding the `appearances_g_batting` index did have a significant impact on the query cost and execution time.<br/>
E. Adding the `salary_idx` index did not have a significant impact on the query execution time and cost.<br/>
F. Adding the `salary_idx` index did have a significant impact on the query execution time, but not the cost.<br/>
G. Adding the `salary_idx` index did have a significant impact on the query cost, but not the execution time.<br/>
H. Adding the `salary_idx` index did have a significant impact on the query cost and execution time.

**Note:** Your answer should have the format like this if you think A and B are both true: `q5b = ['A', 'B']`. The autograder is **case sensitive** but order should not matter.

In [78]:
q5d = ['D','E']

In [79]:
grader.check("q5d")

q5d results: All test cases passed!

<!-- BEGIN QUESTION -->

### Question 5di Justification

1. **Explain your answer to Question 5d** above based on your knowledge from lectures and details from inspecting the query plans.
2. If there were any options you did NOT select, **choose any one of them and explain why you did not select it.**

Your answer should be **no longer than 3 sentences.** **Explicitly state which answer option(s) you chose or didn't choose** in addition to your explanations. For example, you could write, "I chose (A) because..." or "I did NOT choose (B) because..."

I chose (D) because adding the appearances_g_batting index significantly reduced both the execution time (from 17.502 ms to 8.731 ms) and cost (from 3563.80 to 2299.47). I chose E because adding the salary_idx index did not significantly impact the execution time or cost, with the query plan showing similar performance before and after its addition. I did not choose B because the appearances_g_batting index affected both execution time and cost, not just the execution time.

<!-- END QUESTION -->

<br/><br/><br/>

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

# Question 6: Indexes, Part 2

Continue the analysis on how indexes impact query peformance.

Before continuing, make sure to drop all existing indexes from previous questions.

In [80]:
# you must run this cell!!!
%sql DROP INDEX IF EXISTS appearances_batting_idx;
%sql DROP INDEX IF EXISTS salary_idx;

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

++
||
++
++

<br><br>

---

## Question 6a

### Question 6ai

Write a query that finds the `player_id`, `year_id`, and `salary` for each player that had played 10 games **and** batted in 10 games (the number of games in which a player played can be found in the `g_all` column of the `appearances` table and the number of games in which a player batted can be found in the `g_batting` column of the `appearances` table). Your query should join the `salaries` and `appearances` table on all the common columns `year_id`, `team_id`, and `player_id`, so feel free to use a natural join.

Your table header should look like this:

| player_id | year_id | salary |
| --- | --- | --- |

4. Finally, record the **cost** and **time** for the sample query. For all questions within this project, we will accept a range of values for your query costs and query timings.

In [81]:
%%sql --save query_6ai result_6ai <<
SELECT player_id, year_id, salary
FROM salaries
NATURAL JOIN appearances
WHERE g_all = 10 AND g_batting = 10;

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

120 rows affected.

In [82]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_6ai = %sqlcmd snippets query_6ai
grading_util.save_results("result_6ai", query_6ai, result_6ai);

result_6ai.DataFrame().head(3)

,player_id,year_id,salary
0,wiggial01,1985,512500.0
1,anderla02,1986,240000.0
2,lakest01,1986,60000.0


In [83]:
grader.check("q6ai")

q6ai results: All test cases passed!

### Question 6aii

Inspect the query plan and record the execution time and cost.

In [84]:
!psql -h localhost -d baseball -c "EXPLAIN ANALYZE {query_6ai}";

                                                                                                    QUERY PLAN                                                                                                     
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Nested Loop  (cost=0.29..3308.72 rows=1 width=21) (actual time=7.422..13.235 rows=120 loops=1)
   ->  Seq Scan on appearances  (cost=0.00..3133.84 rows=22 width=20) (actual time=0.014..10.509 rows=1289 loops=1)
         Filter: ((g_all = 10) AND (g_batting = 10))
         Rows Removed by Filter: 102967
   ->  Index Scan using salaries_pkey on salaries  (cost=0.29..7.95 rows=1 width=28) (actual time=0.002..0.002 rows=0 loops=1289)
         Index Cond: ((year_id = appearances.year_id) AND ((team_id)::text = (appearances.team_id)::text) AND ((lg_id)::text = (appearances.l

In [85]:
result_6aii_cost = 3308.72
result_6aii_timing = 12.05

In [86]:
grader.check("6aii")

6aii results: All test cases passed!

## Question 6b

### Question 6bi

Write a query that finds the `player_id`, `year_id`, and `salary` for each player that had played 10 games __or__ batted in 10 games. Please use a natural join to join the `salaries` and `appearances` table on all the common columns `year_id`, `team_id`, and `player_id`.

Your table header should look like this:

| player_id | year_id | salary |
| --- | --- | --- |

In [87]:
%%sql --save query_6bi result_6bi <<
SELECT player_id, year_id, salary
FROM salaries
NATURAL JOIN appearances
WHERE g_all = 10 OR g_batting = 10;

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

216 rows affected.

In [88]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_6bi = %sqlcmd snippets query_6bi
grading_util.save_results("result_6bi", query_6bi, result_6bi);
result_6bi.DataFrame().head(3)

,player_id,year_id,salary
0,wiggial01,1985,512500.0
1,forscke01,1986,100000.0
2,carltst01,1986,60000.0


In [89]:
grader.check("q6bi")

q6bi results: All test cases passed!

### Question 6bii

Inspect the query plan and record the execution time and cost.

In [90]:
!psql -h localhost -d baseball -c "EXPLAIN ANALYZE {query_6bi}";

                                                                                                                   QUERY PLAN                                                                                                                   
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Hash Join  (cost=3194.62..3931.39 rows=1 width=21) (actual time=11.822..18.282 rows=216 loops=1)
   Hash Cond: ((salaries.year_id = appearances.year_id) AND ((salaries.team_id)::text = (appearances.team_id)::text) AND ((salaries.lg_id)::text = (appearances.lg_id)::text) AND ((salaries.player_id)::text = (appearances.player_id)::text))
   ->  Seq Scan on salaries  (cost=0.00..459.28 rows=26428 width=28) (actual time=0.010..1.665 rows=26428 loops=1)
   ->  Hash  (cost=3133.84..3133.84 rows=3039 width=20) (actual t

In [91]:
result_6bii_cost = 3930.71
result_6bii_timing = 17.820

In [92]:
grader.check("6bii")

6bii results: All test cases passed!

## Question 6c
Now, let's see the impact of adding an index on the `g_batting` column. Create an index on the `g_batting` column. Re-inspect the queries from `Question 6a` and `Question 6b` and record the respective execution costs and times.

In [93]:
%%sql
DROP INDEX IF EXISTS appearances_g_batting_idx;
CREATE INDEX appearances_g_batting_idx ON appearances(g_batting);


Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

++
||
++
++

In [94]:
# record the updated costs for Question 6a ("and" query)
!psql -h localhost -d baseball -c "EXPLAIN ANALYZE {query_6ai}";

                                                                                                    QUERY PLAN                                                                                                     
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Nested Loop  (cost=19.14..1808.42 rows=1 width=21) (actual time=2.480..4.729 rows=120 loops=1)
   ->  Bitmap Heap Scan on appearances  (cost=18.85..1633.55 rows=22 width=20) (actual time=0.254..2.125 rows=1289 loops=1)
         Recheck Cond: (g_batting = 10)
         Filter: (g_all = 10)
         Rows Removed by Filter: 58
         Heap Blocks: exact=899
         ->  Bitmap Index Scan on appearances_g_batting_idx  (cost=0.00..18.85 rows=1407 width=0) (actual time=0.147..0.147 rows=1347 loops=1)
               Index Cond: (g_batting = 10)
   ->  Index Scan using salarie

In [95]:
result_6c_and_index_cost = 1797.95
result_6c_and_index_timing = 4.282

In [96]:
# record the updated costs for Question 6b ("or" query)
!psql -h localhost -d baseball -c "EXPLAIN ANALYZE {query_6bi}";

                                                                                                                   QUERY PLAN                                                                                                                   
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Hash Join  (cost=3194.62..3931.39 rows=1 width=21) (actual time=12.361..18.869 rows=216 loops=1)
   Hash Cond: ((salaries.year_id = appearances.year_id) AND ((salaries.team_id)::text = (appearances.team_id)::text) AND ((salaries.lg_id)::text = (appearances.lg_id)::text) AND ((salaries.player_id)::text = (appearances.player_id)::text))
   ->  Seq Scan on salaries  (cost=0.00..459.28 rows=26428 width=28) (actual time=0.012..1.650 rows=26428 loops=1)
   ->  Hash  (cost=3133.84..3133.84 rows=3039 width=20) (actual t

In [97]:
result_6c_or_index_cost = 3930.71
result_6c_or_index_timing = 17.429

In [98]:
grader.check("q6c")

q6c results: All test cases passed!

<br/><br/>

---

## Question 6d: Multiple-attribute index

Now, create a multiple column index on `g_batting` and `g_all` called `appearances_g_batting_g_all_idx` and record the query execution time and cost for the "or" command in `Question 6b`.

Before continuing, make sure to drop all existing indexes from previous questions.

In [99]:
# you must run this cell!!!
%sql DROP INDEX IF EXISTS appearances_g_batting_idx;
%sql DROP INDEX IF EXISTS salary_idx;

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

++
||
++
++

In [100]:
%%sql
DROP INDEX IF EXISTS appearances_g_batting_all_idx;
CREATE INDEX appearances_g_batting_g_all_idx ON appearances(g_batting, g_all);

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

++
||
++
++

In [101]:
# record the updated costs for Question 6b ("or" query)
!psql -h localhost -d baseball -c "EXPLAIN ANALYZE {query_6bi}";

                                                                                                                   QUERY PLAN                                                                                                                   
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Hash Join  (cost=2868.53..3605.30 rows=1 width=21) (actual time=3.451..9.937 rows=216 loops=1)
   Hash Cond: ((salaries.year_id = appearances.year_id) AND ((salaries.team_id)::text = (appearances.team_id)::text) AND ((salaries.lg_id)::text = (appearances.lg_id)::text) AND ((salaries.player_id)::text = (appearances.player_id)::text))
   ->  Seq Scan on salaries  (cost=0.00..459.28 rows=26428 width=28) (actual time=0.009..1.655 rows=26428 loops=1)
   ->  Hash  (cost=2807.75..2807.75 rows=3039 width=20) (actual tim

In [102]:
result_6d_multiple_col_index_cost = 3614.65
result_6d_multiple_col_index_timing = 10.071

In [103]:
grader.check("q6d")

q6d results: All test cases passed!

In [104]:
[result_6aii_cost, result_6aii_timing], [result_6bii_cost, result_6bii_timing], [result_6c_and_index_cost, result_6c_and_index_timing], [result_6c_or_index_cost, result_6c_or_index_timing], [result_6d_multiple_col_index_cost, result_6d_multiple_col_index_timing]

([3308.72, 12.05],
 [3930.71, 17.82],
 [1797.95, 4.282],
 [3930.71, 17.429],
 [3614.65, 10.071])

<br/><br/>

---

## Question 6e
Given your findings from inspecting the query plans from all parts of this `Question 6`, assign the variable `q6e` to a list of **ALL** below statements that are true.

A. Adding an index on a column used in an `AND` predicate will reduce the query cost _but not_ the execution time.<br/>
B. Adding an index on a column used in an `AND` predicate will reduce the query cost _and_ the execution time.<br/>
C. Adding an index on a column used in an `OR` predicate will reduce the query cost _but not_ the execution time.<br/>
D. Adding an index on a column used in an `OR` predicate will reduce the query cost _and_ the execution time.<br/>
E. Adding a multicolumn index on columns in an `OR` predicate will reduce the query cost _but not_ the execution time.<br/>
F. Adding a multicolumn index on columns in an `OR` predicate will reduce the query cost _and_ the execution time.

**Note:** Your answer should have the format like this if you think A and B are both true: `q6e = ['A', 'B']`. The autograder is **case sensitive** but order should not matter.

In [105]:
q6e = ['B']

grader.check("q6e")

<!-- BEGIN QUESTION -->

### Question 6ei Justification

1. **Explain your answer to `Question 6e`** above based on your knowledge from lectures and details from inspecting the query plans.
2. If there were any options you did NOT select, **choose any one of them and explain why you did not select it.**

Your answer should be **no longer than 3 sentences** and **explicitly state which answer option(s) you chose or didn't choose** in addition to your explanations. For example, you could write, "I chose (A) because..." or "I did NOT choose (B) because..."

I chose (B) because adding an index on a column used in an AND predicate significantly improves query performance by reducing both query cost and execution time, as the index allows for more efficient access to the required rows. I did NOT choose (C) because while adding an index can reduce the query cost in an OR predicate, it often leads to longer execution times due to the need for combining results from multiple index scans.

<!-- END QUESTION -->

<br/><br/><br/>

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

# Question 7: Indexes and Aggregations, Part 3

Continue the analysis on how indexes impact query performance.

Before continuing, make sure to drop all existing indexes from previous questions.

In [106]:
# you must run this cell!!!
%sql DROP INDEX IF EXISTS appearances_g_batting_idx;
%sql DROP INDEX IF EXISTS salary_idx;
%sql DROP INDEX IF EXISTS appearances_g_batting_all_idx;

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

++
||
++
++

---

## Question 7a

Write 2 queries, one that finds the minimum salary from the salary table `salaries` and one that finds the average. Inspect the queries' query plans and record their execution times and costs.

### Question 7ai

**Find the minimum salary.** Call this column `min_salary`.

Your table header should look like this:

| min_salary |
| --- |

In [107]:
%%sql --save query_7ai result_7ai <<
SELECT MIN(salary) AS min_salary
FROM salaries;

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

1 rows affected.

In [108]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_7ai = %sqlcmd snippets query_7ai
grading_util.save_results("result_7ai", query_7ai, result_7ai);

display(result_7ai)

!psql -h localhost -d baseball -c "EXPLAIN ANALYZE {query_7ai}";

min_salary
0.0


                                                    QUERY PLAN                                                    
------------------------------------------------------------------------------------------------------------------
 Aggregate  (cost=525.35..525.36 rows=1 width=8) (actual time=3.647..3.648 rows=1 loops=1)
   ->  Seq Scan on salaries  (cost=0.00..459.28 rows=26428 width=8) (actual time=0.008..1.584 rows=26428 loops=1)
 Planning Time: 0.314 ms
 Execution Time: 3.703 ms
(4 rows)



In [109]:
result_7ai_query_cost = 526.36
result_7ai_query_timing = 3.495

In [110]:
grader.check("q7ai")

q7ai results: All test cases passed!

### Question 7aii

**Find the average salary.** Call this column `average_salary`.

Your table header should look like this:

| average_salary |
| --- |

In [111]:
%%sql --save query_7aii result_7aii <<
SELECT AVG(salary) AS average_salary
FROM salaries;

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

1 rows affected.

In [112]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_7aii = %sqlcmd snippets query_7aii
grading_util.save_results("result_7aii", query_7aii, result_7aii);

display(result_7aii)

!psql -h localhost -d baseball -c "EXPLAIN ANALYZE {query_7aii}";

average_salary
2085634.053125473


                                                    QUERY PLAN                                                    
------------------------------------------------------------------------------------------------------------------
 Aggregate  (cost=525.35..525.36 rows=1 width=8) (actual time=4.029..4.030 rows=1 loops=1)
   ->  Seq Scan on salaries  (cost=0.00..459.28 rows=26428 width=8) (actual time=0.009..1.604 rows=26428 loops=1)
 Planning Time: 0.424 ms
 Execution Time: 4.096 ms
(4 rows)



In [113]:
result_7aii_query_cost = 525.36
result_7aii_query_timing = 4.157

In [114]:
grader.check("q7aii")

q7aii results: All test cases passed!

<br><br>

---
## Question 7b
Create an index on the `salary` column in the `salaries` table and re-inspect the query plans from the previous part and record the respective execution time and cost.

In [115]:
%%sql
DROP INDEX IF EXISTS salary_idx;
CREATE INDEX salary_idx ON salaries(salary);


Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

++
||
++
++

In [116]:
# record the updated costs for "min" query
!psql -h localhost -d baseball -c "EXPLAIN ANALYZE {query_7ai}";

                                                                  QUERY PLAN                                                                  
----------------------------------------------------------------------------------------------------------------------------------------------
 Result  (cost=0.32..0.33 rows=1 width=8) (actual time=0.047..0.048 rows=1 loops=1)
   InitPlan 1 (returns $0)
     ->  Limit  (cost=0.29..0.32 rows=1 width=8) (actual time=0.044..0.045 rows=1 loops=1)
           ->  Index Only Scan using salary_idx on salaries  (cost=0.29..762.78 rows=26428 width=8) (actual time=0.043..0.043 rows=1 loops=1)
                 Index Cond: (salary IS NOT NULL)
                 Heap Fetches: 0
 Planning Time: 0.426 ms
 Execution Time: 0.109 ms
(8 rows)



In [117]:
result_7b_min_query_cost = 0.33
result_7b_min_query_timing = 0.052

In [118]:
# record the updated costs for "avg" query
!psql -h localhost -d baseball -c "EXPLAIN ANALYZE {query_7aii}";

                                                    QUERY PLAN                                                    
------------------------------------------------------------------------------------------------------------------
 Aggregate  (cost=525.35..525.36 rows=1 width=8) (actual time=4.011..4.012 rows=1 loops=1)
   ->  Seq Scan on salaries  (cost=0.00..459.28 rows=26428 width=8) (actual time=0.009..1.594 rows=26428 loops=1)
 Planning Time: 0.352 ms
 Execution Time: 4.075 ms
(4 rows)



In [119]:
result_7b_avg_query_cost = 526.36
result_7b_avg_query_timing = 4.086

In [120]:
grader.check("q7b")

q7b results: All test cases passed!

<!-- BEGIN QUESTION -->

<br><br>

---

## Question 7c
Given your findings from `Question 7`, which of the following statements is true? **Select one.**

<br> A. An index on the column being aggregated in a query will always provide a performance enhancement.
<br> B. A query finding the `MIN(salary)` will always benefit from an index on salary, but a query finding `MAX(salary)` will not.
<br> C. A query finding the `COUNT(salary)` will always benefit from an index on salary, but a query finding `AVG(salary)` will not.
<br> D. Queries finding the `MIN(salary)` or `MAX(salary)` will always benefit from an index on salary, but queries finding `AVG(salary)` or `COUNT(salary)` will not.

**State and justify your answer.**

1. **Explain your answer** based on your knowledge from lectures and details of the query plans.
2. **Of the answer options you did not select, choose any one of them and explain why that option is wrong.**

Your response should be no longer than 3 sentences.
 
**Note:** Your answer should be formatted as follows: `(A) because ...` and `Not (A) because ...`

(D) because queries finding the MIN(salary) or MAX(salary) can utilize the index for faster access to the extreme values in the ordered dataset, which is especially beneficial. Not (D) because queries finding AVG(salary) and COUNT(salary) require scanning all values in the dataset, which generally results in higher execution costs. For example, the average salary query cost of 526.36 compared to the much lower cost of finding the minimum salary.

<!-- END QUESTION -->

<br/><br/><br/>

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

# Question 8: Clustered Indexes
In this question, we will inspect the impact that clustering our data on an index can have on a query's performance.

Before continuing, make sure to drop all existing indexes from previous questions.

In [121]:
# you must run this cell!!!
%sql DROP INDEX IF EXISTS appearances_g_batting_idx;
%sql DROP INDEX IF EXISTS salary_idx;
%sql DROP INDEX IF EXISTS appearances_g_batting_all_idx;

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

++
||
++
++

---

## Question 8a

### Question 8ai

Write a query that finds the `player_id`, `year_id`, `team_id`, and `ab` for all players whose `ab` was above 500. Inspect the query plan and record the execution time and cost. Your table header should look like this:

| player_id | year_id | team_id | ab |
| --- | --- | --- | --- |

Optional: AB, short for ["At bat"](https://en.wikipedia.org/wiki/At_bat), is a baseball player statistic.

In [122]:
%%sql --save query_8ai result_8ai <<
SELECT player_id, year_id, team_id, ab
FROM batting
WHERE ab > 500;

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

8839 rows affected.

In [123]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_8ai = %sqlcmd snippets query_8ai
grading_util.save_results("result_8ai", query_8ai, result_8ai);
result_8ai.DataFrame().head(3)

,player_id,year_id,team_id,ab
0,dalryab01,1884,CHN,521
1,hornujo01,1884,BSN,518
2,ansonca01,1886,CHN,504


In [124]:
grader.check("q8ai")

q8ai results: All test cases passed!

### Question 8aii

Inspect the query plan and record the execution time and cost.

In [125]:
!psql -h localhost -d baseball -c "EXPLAIN ANALYZE {query_8ai}";

                                                 QUERY PLAN                                                 
------------------------------------------------------------------------------------------------------------
 Seq Scan on batting  (cost=0.00..2884.05 rows=8738 width=21) (actual time=0.219..11.085 rows=8839 loops=1)
   Filter: (ab > 500)
   Rows Removed by Filter: 95485
 Planning Time: 0.292 ms
 Execution Time: 11.378 ms
(5 rows)



In [126]:
result_8aii_cost = 2884.05
result_8aii_timing = 9.743

In [127]:
grader.check("8aii")

8aii results: All test cases passed!

<br><br>

---

## Question 8b

Cluster the `batting` table on its primary key (Hint: use the psql meta-command `\di` or `\d batting` to find out what name of the primary key is). We are able to directly cluster on the primary key (without first creating a separate index) because Postgres automatically creates an index for it.

Then, re-inspect the query plan for the query from `Question 8a` and record the execution time and cost.

In [128]:
%%sql
CLUSTER batting USING batting_pkey;

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

++
||
++
++

In [129]:
# check the updated costs for query in Question 8a
!psql -h localhost -d baseball -c "EXPLAIN ANALYZE {query_8ai}";

                                                 QUERY PLAN                                                 
------------------------------------------------------------------------------------------------------------
 Seq Scan on batting  (cost=0.00..2878.05 rows=8738 width=21) (actual time=0.022..11.751 rows=8839 loops=1)
   Filter: (ab > 500)
   Rows Removed by Filter: 95485
 Planning Time: 0.313 ms
 Execution Time: 12.090 ms
(5 rows)



In [130]:
result_8b_cost = 2878.05
result_8b_timing = 12.341

In [131]:
grader.check("q8b")

q8b results: All test cases passed!

<br><br>

---

## Question 8c

Now, let's try clustering the table based on another index. Create an index on `ab` column called `ab_idx` in the `batting` table AND cluster the `batting` table with this new index. Now, re-inspect the query plan and record the execution time and cost.

In [132]:
%%sql --save query_8c result_8c <<
DROP INDEX IF EXISTS ab_idx;
CREATE INDEX ab_idx ON batting (ab);

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

In [133]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_8c = %sqlcmd snippets query_8c
grading_util.save_results("result_8c", query_8c, result_8c);

# check the updated costs for query in Question 8a
!psql -h localhost -d baseball -c "EXPLAIN ANALYZE {query_8ai}";

                                                      QUERY PLAN                                                      
----------------------------------------------------------------------------------------------------------------------
 Bitmap Heap Scan on batting  (cost=100.01..1783.24 rows=8738 width=21) (actual time=0.701..3.956 rows=8839 loops=1)
   Recheck Cond: (ab > 500)
   Heap Blocks: exact=1311
   ->  Bitmap Index Scan on ab_idx  (cost=0.00..97.83 rows=8738 width=0) (actual time=0.531..0.531 rows=8839 loops=1)
         Index Cond: (ab > 500)
 Planning Time: 0.380 ms
 Execution Time: 4.301 ms
(7 rows)



In [134]:
result_8c_cost = 1792.87
result_8c_timing = 4.024

In [135]:
grader.check("q8c")

q8c results: All test cases passed!

<br><br>

---

## Question 8d
Given your findings from inspecting the query plans from Questions 8a, 8b, and 8c, assign the variable `q8d` to a list of **ALL** statements that are true.

A. Clustering based on the `ab_idx` decreased the cost of the query.<br/>
B. Clustering based on the `ab_idx` increased the cost of the query.<br/>
C. Clustering based on the `ab_idx` increased the execution time of the query.<br/>
D. Clustering based on the `ab_idx` decreased the execution time of the query.<br/>
E. Clustering based on the `batting_pkey` decreased the cost of the query.<br/>
F. Clustering based on the `batting_pkey` increased the cost of the query.<br/>
G. Clustering based on the `batting_pkey` increased the execution time of the query.<br/>
H. Clustering based on the `batting_pkey` decreased the execution time of the query.<br/>
I. None of the above
    
**Note:** Your answer should have the format like this if you think A and B are both true: `q8d = ['A', 'B']`. The autograder is **case sensitive** but order should not matter.

In [136]:
q8d = ['A','D','G']

In [137]:
grader.check("q8d")

q8d results: All test cases passed!

<!-- BEGIN QUESTION -->

<br><br>

---

### Question 8di Justification

1. **Explain your answer to `Question 8d`** above based on your knowledge from lectures and details from inspecting the query plans.
2. If there were any options you did NOT select, **choose any one of them and explain why you did not select it.**

Your answer should be **no longer than 3 sentences** and **explicitly state which answer option(s) you chose or didn't choose** in addition to your explanations. For example, you could write, "I chose (A) because..." or "I did NOT choose (B) because..."

I chose (A) because clustering the batting table based on the ab_idx index resulted in a lower execution cost, indicating improved access to the data for the query that filters for ab > 500. I also chose (D) because the execution time for the same query decreased after clustering on ab_idx, demonstrating enhanced performance. I chose (G) because clustering based on the batting_pkey increased the execution time of the query, indicating that there was a decrease in execution performance. I did NOT choose (C) because the execution time did not increase, it rather improved due to better data organization after clustering on the index.

<!-- END QUESTION -->

<br/><br/><br/>

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

# Question 9: Cost of Index Management
Until now, we have seen the positive potential impact that indexes can have on query performance, but remember in real world technologies/applications, we will be routinely receiving new data (and in large quantities) which would trigger regular updates to our tables. In this section, we will dive into the cost of managing the indexes that we create.

Before starting this question, be sure to delete any indexes by running the below cell.

In [138]:
# you must run this cell!!!
%sql DROP INDEX IF EXISTS appearances_g_batting_idx;
%sql DROP INDEX IF EXISTS salary_idx;
%sql DROP INDEX IF EXISTS appearances_g_batting_all_idx;
%sql DROP INDEX IF EXISTS ab_idx;

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

++
||
++
++

---

## Question 9a

Record the time it takes to insert 300,000 rows into the `salaries` table when no additional index is configured.

Run the following cell to setup a column to track which rows we added as part of these inserts.

In [139]:
%sql ALTER TABLE salaries ADD added boolean DEFAULT False;

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

++
||
++
++

Next, run the provided update script and record the **wall time** (found in the 2nd line of output).

**NOTE:** Running the below cell multiple times may result in an error, unless you first delete the rows with the cell given at the end of this subpart.

In [140]:
%%time
%%sql
DO $$
 DECLARE counter INTEGER := 1;
 BEGIN
     FOR counter IN 100001..400000 LOOP
     INSERT INTO salaries (year_id, team_id, lg_id, player_id, salary, added)
         VALUES (2021, 'ATL', 'NL', 'p' || counter, RANDOM() * 1000000, true);
     END LOOP;
END;
$$;

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

CPU times: user 9.88 ms, sys: 29 µs, total: 9.91 ms
Wall time: 2.98 s


In [141]:
result_9a_timing = 2.91

In [142]:
grader.check("q9a")

q9a results: All test cases passed!

<br/><br/>

**Before moving onto the next question**,  delete all the rows that were added to the table from the update script.

In [143]:
%%sql
/* just run this cell */
DELETE FROM salaries
WHERE added = 'true';

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

300000 rows affected.

++
||
++
++

<br><br>

---

## Question 9b

Now, create an index on the `salary` column and record the **wall time** after executing the update script. Make sure to first run the previous cell to rollback any changes from the previous part!

In [144]:
%%sql
DROP INDEX IF EXISTS salary_idx;
CREATE INDEX salary_idx ON salaries(salary);

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

++
||
++
++

**NOTE:** Running the below cell multiple times may result in an error, unless you first delete the rows with the cell given at the end of last subpart.

In [145]:
%%time
%%sql
DO $$
 DECLARE counter INTEGER := 1;
 BEGIN
     FOR counter IN 100001..400000 LOOP
     INSERT INTO salaries (year_id, team_id, lg_id, player_id, salary, added)
         VALUES (2021, 'ATL', 'NL', 'p' || counter, RANDOM() * 1000000, true);
     END LOOP;
END;
$$;

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

CPU times: user 9.29 ms, sys: 971 µs, total: 10.3 ms
Wall time: 5.23 s


In [146]:
result_9b_timing = 4.94

In [147]:
grader.check("q9b")

q9b results: All test cases passed!

<!-- BEGIN QUESTION -->

<br><br>

---

## Question 9c
What difference did you notice when you added an index into the salaries table and re-timed the update? Why do you think it happened? Your answer should be no longer than 3 sentences.

When we added an index to the salaries table, we noticed that the time it took to update actually went up a lot compared to when there was no index. This happens because, while indexes help speed up reading data, they also make writing a bit slower. Every time a new row is added, the database has to update the index too, which takes extra time. So, in this case the extra time for updating the index ended up being more than the time saved during data retrieval.

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

<br/><br/><br/>

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

# Question 10: Project Takeaways

In this project, we explored how the database system optimizes query execution and how users can futher tune the performance of their queries.

Familiarizing yourself with these optimization and tuning methods will make you a better data engineer. In this question, we'll ask you to recall and summarize these concepts. Who knows? Maybe one day it will help you during an interview or on a project.

In the following answer cell,
1. Name 3 non-trivial, **distinctly different** methods to optimize query performance that you learned in this project. Some places to start looking are some of the question topics.
2. For each method, summarize how and why it can optimize query performance. Feel free to discuss any drawbacks, if applicable.

Your entire answer should be no longer than ten sentences. Each method identification/discussion is 2 points.


Some key takeaways i had for this project was that there are alot of methods to fine tune queries for better performance and some methods are faster than others for specific scenarios. 3 distinctly different methods I found were the following:  
Indexing: Creating indexes on columns frequently used in WHERE clauses, JOIN conditions, or ORDER BY clauses can significantly speed up query execution. Indexes allow the database to locate rows more efficiently, reducing the amount of data that needs to be scanned. However, maintaining indexes can incur overhead during INSERT, UPDATE, and DELETE operations, potentially slowing down these actions.

Clustering: Clustering a table based on an index changes how the data is physically stored to match the index order, which speeds up range queries. For example, clustering on a primary key helps when you're pulling data for a specific range of keys. The downside, however, is that clustering can be very resource-heavy and isn't great for tables that change a lot, since you have to keep the physical order aligned with the logical order.

Partitioning: Partitioning is dividing a big table into smaller chunks so it’s easier to manage. This helps speed things up because the database can focus on just the relevant parts instead of scanning the whole table every time. It’s especially handy for queries that deal with specific time frames or categories. However, it can also make things a bit more complicated to set up and maintain, and if you don’t do it right, it might not help as much as you’d hope.

<!-- END QUESTION -->

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

# Congratulations! You have finished Project 2.

Run the following cell to zip and download the results of your queries. You will also need to run the export cell at the end of the notebook.

**Please save your notebook before exporting (this is a good time to do it!)** Otherwise, we may not be able to export your written responses to `proj2.pdf`. We will not be accepting regrade requests for failure to render written responses.

**For your submission on Gradescope, you will only need to submit the single `proj2.zip` file generated by the export cell.** Please ensure that your submission `proj2.zip` file includes `proj2.pdf`, `proj2.ipynb`, and `results.zip`. 

**Please ensure that public tests pass upon submission.** It is your responsibility to wait until the autograder finishes running. We will not be accepting regrade requests for submission issues.

**Common submission issues:** You MUST submit the generated zip file to the autograder. However, Safari is known to automatically unzip files upon downloading. You can fix this by going into Safari preferences, and deselect the box with the text "Open safe files after downloading" under the "General" tab. If you experience issues with downloading via clicking on the link, you can also navigate to the project 2 directory within JupyterHub (remove `proj2.ipynb` from the url), and manually download the generated zip files. Please post on Ed if you encounter any other submission issues.

Run the following cell to zip and download the results of your queries. You will also need to run the export cell at the end of the notebook.

In [ ]:
grading_util.prepare_submission_and_cleanup()

In [ ]:
# Close SQL magic connection
# You may disregard "RunTimeError: Could not close connection"
# %sql --close postgresql://127.0.0.1:5432/baseball

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(files=['results.zip'])